## Main Notebook for Training and Evaluation

---
> Evangelia P. Panourgia, Master Student in Data Science, AUEB <br />
> Department of Informatics, Athens University of Economics and Business <br />
> eva.panourgia@aueb.gr <br/><br/>


### Install Libraries

In [2]:
!pip install nltk optuna xgboost

### Setting the Scene 
- We will import all the needeed libraries.

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import random
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.multiclass import OneVsRestClassifier
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import os
import joblib  # For saving and loading models
from sklearn.metrics import classification_report
from optuna import TrialPruned
from sklearn.model_selection import cross_val_score
from optuna.exceptions import TrialPruned
import os
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /Users/evangelia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evangelia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/evangelia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data 
- We will load the preprocessed data (`data_augmented__nlp_incidents_train.csv`) being pre-processed with `data augmentes` (generation of synthetic data usage synonyms) and basic nlp preprocess.
- Furthermore, we will load the unlabeleed data of the competition in ordeer to predict them (`incidents.csv`).

In [3]:
df_augmented= pd.read_csv('data/data_augmented__nlp_incidents_train.csv') # load data after data augmentation
testset_competition = pd.read_csv('data/incidents.csv', index_col=0) # load testing data (conception phase, unlabeled):

,year,month,day,country,title,text
0,1994,5,5,us,Recall Notification: FSIS-017-94,Case Number: 017-94 \n Date Opene...
1,1994,5,12,us,Recall Notification: FSIS-048-94,Case Number: 048-94 \n Date Opene...
2,1995,4,16,us,Recall Notification: FSIS-032-95,Case Number: 032-95 \n Date Opene...
3,1998,7,16,ca,Archive - ALLERGY ALERT -- PRESENCE OF UNDECLA...,PRESENCE OF UNDECLARED NUTS IN ORIGINALE AUGUS...
4,1998,8,6,us,Recall Notification: FSIS-018-98,Case Number: 018-98 Recall Notification Repor...
...,...,...,...,...,...,...
560,2022,6,29,au,The Fresh Salad Co Thai Coconut Wild Rice Prep...,Page Content ​ ​​​​ ​Date publ...
561,2022,7,18,au,Powered by Plants Pty Ltd — Cleanfit Plant Pro...,PRA number 2022/19525 Published date 18 Jul 20...
562,2022,7,20,ca,Certain Enjoy Life brand Soft Baked Cookies – ...,Food recall warning Certain Enjoy Life brand S...
563,2022,7,28,hk,Imported biscuit may contain allergen (peanuts),Imported biscuit may contain allergen (peanuts...


In [4]:
df_augmented = df_augmented[['title','text','hazard-category','product-category','hazard','product']]
df_augmented.head(3) # preview preproccessed data 

,title,text,hazard-category,product-category,hazard,product
0,recal notif fsis-024-94,case number 024-94 date open 07/01/1994 date c...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage
1,recal notif fsis-033-94,case number 033-94 date open 10/03/1994 date c...,biological,"meat, egg and dairy products",listeria spp,sausage
2,recal notif fsis-014-94,case number 014-94 date open 03/28/1994 date c...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices
3,recal notif fsis-009-94,case number 009-94 date open 03/04/1994 date c...,foreign bodies,"meat, egg and dairy products",plastic fragment,thermal processed pork meat
4,recal notif fsis-001-94,case number 001-94 date open 01/07/1994 date c...,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast
...,...,...,...,...,...,...
13594,cf barrack world eat_up genial transfus dome i...,atomic_number_98 root_on world devour varieti ...,chemical,other food product / mixed,unauthorised substance ethylene oxide,ramen noodles
13595,inspir public non take_in varieti cup attic sp...,cf itch world eat_up genial transfus bean impo...,chemical,other food product / mixed,unauthorised substance ethylene oxide,ramen noodles
13596,cystic_fibrosi inspir world non wast sort tran...,pancreatic_fibrosi pep_up populac go_through s...,chemical,other food product / mixed,unauthorised substance ethylene oxide,ramen noodles
13597,cystic_fibrosi urg world non ware varieti cup ...,fibrocystic_disease_of_the_pancrea urg populac...,chemical,other food product / mixed,unauthorised substance ethylene oxide,ramen noodles


In [ ]:
testset_competition.head(3)# preview test data 

## Benchmarks Analysis 
- Benchmark analysis is crucial for evaluating classification performance in multiclass imbalance settings because it provides reference points for how well your model is performing relative to simple baseline classifiers. The `Random Classifier` and `Majority Classifier` are commonly used as benchmarks for the following reasons:

### Random Classifier 
- A Random Classifier predicts class labels randomly, with **uniform** based on the distribution of classes. It sets a minimal baseline and helps understand:

- `Baseline Performance`: This represents the expected performance `without learning from the data`. `If a model performs worse than a random classifier, it indicates either issues in the model or unsuitable features`.

- `Chance Levels`: It shows what performance you'd `get by chance alone`, especially useful for imbalanced datasets where metrics like accuracy can be misleading.


### Majority Classifier

- A Majority Classifier always **predicts the majority class** (`the class with the highest frequency in the training data`). 

- It helps understand:

    - `Handling Imbalance`: In multiclass imbalanced datasets, accuracy can be dominated by the majority class. The majority classifier provides a baseline to compare how well your model captures minority classes.
    - `Baseline of Naïve Solutions`: The majority classifier reflects the simplest possible rule for prediction. If a model's performance is close to that of a majority classifier, it suggests the model is failing to generalize or adapt to the minority classes.
    - `Focus on Class Imbalance`: Metrics like weighted accuracy, balanced accuracy, or macro-F1 score should be significantly better than those achieved by the majority classifier to indicate that a model is addressing imbalance effectively.

- Note in the following code cell I implement the code for Random and Majority Classifier, in order to have a high level of "logic" we added the split steps of trainingtest set, but for example for the Random Classifier it is useless as it is not affected from the input, dont learn from data.
    - Hoever, this "skeleton" is useful for the reamaining algorythms to buils in (both traditional and advanced) 

- More specifically, 

    - Random Classifier  Effect of X: The X values (features) **do not influence the random classifier's predictions**. It does not learn from the data in the feature column. Its predictions are purely random, so changing X will not alter its performance.
    - Majority Classifier Effect of X: The feature column X is ignored by the majority classifier, as it does not use features for prediction. Instead, it looks only at the distribution of y in the training data.

### Regarding the Implementation 
- The `DummyClassifier in scikit-learn` is a baseline model designed to evaluate classification algorithms by comparing them against simplistic strategies. These strategies provide minimal logic to make predictions and are often used as benchmarks to understand how well a more complex model performs.
    - `strategy="uniform"` (for Random Classifier): 
        - Predicts a class randomly and uniformly across all possible classes.
        - Each class has an equal probability of being selected, irrespective of the class distribution in the training data.
        - Use Case: Ideal for scenarios where you want to simulate random guessing.
    - `strategy="most_frequent"` (for Majoriry Classification)
        - Always predicts the most frequent class observed in the training data.
        - Ignores the input features entirely and focuses only on the training set's class distribution.
        - Use Case: Useful for understanding how well a naive baseline would perform if you simply predicted the majority class.

In [5]:
def evaluate_baselines(dataframe, feature_column):
    """
    Function to evaluate random and majority classifiers on a given dataframe.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.
    """
    np.random.seed(42)  # For reproducibility

    # Train-test split with optional stratification
    trainset, testset = train_test_split(
        dataframe, 
        test_size=0.2, 
        random_state=2024, 
        # "skeleton" for the main algo here add stratisfy to hold proportion of classes 
    )
   
    # Random and Majority classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Evaluating for label: {label}")

        # Features and target
        X_train = trainset[feature_column]
        y_train = trainset[label]
        X_test = testset[feature_column]
        y_test = testset[label]

        # Random Classifier
        random_clf = DummyClassifier(strategy="uniform", random_state=2024)
        random_clf.fit(X_train, y_train) # it is uselless X stimulate the logic of a real algo. 
        testset['predictions-random-' + label] = random_clf.predict(X_test)

        # Majority Classifier
        majority_clf = DummyClassifier(strategy="most_frequent")
        majority_clf.fit(X_train, y_train)# it is uselless X stimulate the logic of a real algo. 
        testset['predictions-majority-' + label] = majority_clf.predict(X_test)

        # Compute F1 scores
        random_f1 = f1_score(y_test, testset['predictions-random-' + label], average='macro', zero_division=0)
        majority_f1 = f1_score(y_test, testset['predictions-majority-' + label], average='macro', zero_division=0)

        print(f"F1 Score for Random Classifier ({label}): {random_f1:.3f}")
        print(f"F1 Score for Majority Classifier ({label}): {majority_f1:.3f}")

        # Generate and save classification reports
        os.makedirs('reports/random', exist_ok=True)
        os.makedirs('reports/majority', exist_ok=True)

        random_report = classification_report(y_test, testset['predictions-random-' + label], zero_division=0)
        majority_report = classification_report(y_test, testset['predictions-majority-' + label], zero_division=0)

        with open(f'reports/random/random_classifier_report_{label}.txt', 'w') as random_file:
            random_file.write(f"Classification Report for Random Classifier ({label}):\n")
            random_file.write(random_report)

        with open(f'reports/majority/majority_classifier_report_{label}.txt', 'w') as majority_file:
            majority_file.write(f"Classification Report for Majority Classifier ({label}):\n")
            majority_file.write(majority_report)
        
        
    
    # Custom metric score calculation
    def compute_score(hazards_true, products_true, hazards_pred, products_pred):
        """
        Custom scoring function to compute the macro F1 score for hazards and products.
        
        Args:
            hazards_true: Ground truth labels for hazards.
            products_true: Ground truth labels for products.
            hazards_pred: Predicted labels for hazards.
            products_pred: Predicted labels for products.
        
        Returns:
            A float representing the combined macro F1 score.
        """
        f1_hazards = f1_score(hazards_true, hazards_pred, average='macro', zero_division=0)
        f1_products = f1_score(
            products_true[hazards_pred == hazards_true],
            products_pred[hazards_pred == hazards_true],
            average='macro', 
            zero_division=0
        )
        return (f1_hazards + f1_products) / 2.

    # Example of calculating scores for Sub-Tasks (if needed):
    # Uncomment the following lines to compute scores for tasks
    print(f"Score Sub-Task 1 - Random Classifier: {compute_score(testset['hazard-category'], testset['product-category'], testset['predictions-random-hazard-category'], testset['predictions-random-product-category']):.3f}")
    print(f"Score Sub-Task 2 - Random Classifier: {compute_score(testset['hazard'], testset['product'], testset['predictions-random-hazard'], testset['predictions-random-product-category']):.3f}")
    print(f"Score Sub-Task 1 - Majority Classifier: {compute_score(testset['hazard-category'], testset['product-category'], testset['predictions-majority-hazard-category'], testset['predictions-majority-product-category']):.3f}")
    print(f"Score Sub-Task 2 - Majority Classifier: {compute_score(testset['hazard'], testset['product'], testset['predictions-majority-hazard'], testset['predictions-majority-product']):.3f}")

# Call the function with the required dataframe (e.g., df_augmented or any other dataframe)
evaluate_baselines(df_augmented, feature_column='text', stratify_column='hazard-category')
# Uncomment the following line to use a different feature column
# evaluate_baselines(df_augmented, feature_column='title', stratify_column='hazard-category')


Evaluating for label: hazard-category
F1 Score for Random Classifier (hazard-category): 0.076
F1 Score for Majority Classifier (hazard-category): 0.047
Evaluating for label: product-category
F1 Score for Random Classifier (product-category): 0.034
F1 Score for Majority Classifier (product-category): 0.019
Evaluating for label: hazard
F1 Score for Random Classifier (hazard): 0.005
F1 Score for Majority Classifier (hazard): 0.001
Evaluating for label: product
F1 Score for Random Classifier (product): 0.000
F1 Score for Majority Classifier (product): 0.000
Score Sub-Task 1 - Random Classifier: 0.057
Score Sub-Task 2 - Random Classifier: 0.003
Score Sub-Task 1 - Majority Classifier: 0.031
Score Sub-Task 2 - Majority Classifier: 0.001


###  Results and Observations
- Label: `hazard-category`
    - Random Classifier F1: 0.076
    - Majority Classifier F1: 0.047
- Performance is slightly better for the Random Classifier, but both are low, indicating the dataset is likely imbalanced, and random guessing doesn't align well with true labels.

- Label: `product-category`
    - Random Classifier F1: 0.034
    - Majority Classifier F1: 0.019
- Performance drops further here. It suggests more complexity or higher imbalance in this label.

- Label: `hazard`
    - Random Classifier F1: 0.005
    - Majority Classifier F1: 0.001
    - Both scores are extremely low, possibly due to:
        - Large number of classes.
        - Sparse distribution of classes.
        - Poor representation of these classes in the Random Classifier's uniform predictions or Majority Classifier's mode.

- Label: `product`
    - Random Classifier F1: 0.000
    - Majority Classifier F1: 0.000
    - Both classifiers completely fail to capture meaningful patterns for this label. This could suggest extreme imbalance or lack of meaningful correlation in the dataset.

- Sub-Tasks
    - Score Sub-Task 1: hazard-category & product-category
        - Random Classifier Score: 0.057
        - Majority Classifier Score: 0.031
        - Indicates the overall performance when combining macro F1 scores for hazard-category and product-category. Random guessing outperforms predicting the most frequent class, but both are weak.
    - Score Sub-Task 2: hazard & product
        - Random Classifier Score: 0.003
        - Majority Classifier Score: 0.001
        - Reflects the severe challenge for these labels. The performance is near zero, affirming the labels require more sophisticated approaches.

-  `Conclusions` : 
- Baseline as a Benchmark:

    - **The poor F1 scores highlight the challenging nature of the task and dataset**.
    - These results provide a benchmark to evaluate future models. Any model achieving significantly higher F1 scores would demonstrate effective learning.

- Dataset Imbalance:

    - The low performance of the Majority Classifier indicates severe class imbalance across all labels.
    - Future models should address this using strategies like stratified sampling, oversampling, or weighted loss functions.

- Complexity of Labels:

    - The complexity increases from hazard-category and product-category to hazard and product, as reflected in the declining F1 scores.

- Actionable Insights:

    - Preprocessing: Investigate the class distributions and apply balancing techniques.
    - Feature Engineering: Consider enhancing the feature column (e.g., using embeddings).
    - Advanced Models: Apply models capable of handling imbalance, such as tree-based methods, ensemble models, or neural networks.


### Traditional and Advanced Approach - Design Decision (skeleton of function) and Limitations in Implemeentation (Resourses) 

- For the traditional ML approach we will run `Logistic regression` and for advanced algorythm `X-Boost` 
- This is because these approaches reflect diffrent things. 
    - `Logistic Regression`:  serves as a simple, interpretable way for understanding how well your data can be modeled with `linear relationships`.
    - `X-Boost`: if XGBoost significantly outperforms logistic regression, it suggests that your data has complex patterns that require `non-linear modeling`.

-  Skeleton Function, we spent time on writting a "skeletton" of function in order to be easy adaptable if we will change the model. Both for logistic regression and x-boost we have the following logic in code : 
    1. <br> split train test stratisfy 
    2. <br> one vs all 
    3. <br> opttuna C regularization  cross - vall 
    4. <br> tf-idf pipeline 
    5. <br>
    6. <br>

Key Note : Due to time - space limitations we  Hyperparameter tuning 

- Limitations 
    1. SMOTE <br>
    2. CROSS VAL <br>
    3. <br>
    4. <br>
    5. <br>
    6. <br> 
- Optuna k-0fold

sklearn  not gpu usage 

logistic not svm due to resource more complex 

smote xoros ayjanei 

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, classification_report
import numpy as np
import os

def evaluate_log_regression_with_ova(dataframe, feature_column, testset_competition):
    """
    Function to evaluate one-vs-all logistic regression classifier on a given dataframe
    without applying SMOTE.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.
        test_competition: The unlabeled dataframe 
    """
    np.random.seed(42)  # For reproducibility

    # Dictionary to store predictions for each label
    predictions_dict = {}

    # Define the pipeline
    text_clf_lr = Pipeline([
        ('vect', TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5)),
        ('clf', LogisticRegression(max_iter=1000, random_state=2024))
    ])

    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Evaluating for label: {label}")

        # Train-test split with stratification based on the current label
        trainset, testset = train_test_split(
            dataframe,
            test_size=0.2,
            random_state=2024,
            stratify=dataframe[label]
        )

        # Extract train and test features
        X_train = trainset[feature_column]
        X_test = testset[feature_column]

        # Target
        y_train = trainset[label]
        y_test = testset[label]

        # Logistic Regression Classifier with OvA using OneVsRestClassifier
        logreg_clf = OneVsRestClassifier(text_clf_lr)
        logreg_clf.fit(X_train, y_train)
        predictions = logreg_clf.predict(X_test)

        # Store predictions separately
        predictions_dict[label] = {
            "y_test": y_test,
            "predictions": predictions
        }

        # Predict unlabelled data **************************************************
        testset_competition[label] = logreg_clf.predict(testset_competition['title'])

        # Compute F1 scores
        logreg_f1 = f1_score(y_test, predictions, average='macro', zero_division=0)
        print(f"F1 Score for Logistic Regression Classifier ({label}): {logreg_f1:.3f}")

        # Generate and save classification reports
        os.makedirs('reports/logreg', exist_ok=True)

        logreg_report = classification_report(y_test, predictions, zero_division=0)

        with open(f'reports/logreg/logreg_classifier_report_{label}.txt', 'w') as logreg_file:
            logreg_file.write(f"Classification Report for Logistic Regression Classifier ({label}):\n")
            logreg_file.write(logreg_report)
            logreg_file.write(f"F1 Score for Logistic Regression Classifier ({label}): {logreg_f1:.3f}")

    # Custom metric score calculation
    def compute_score(hazards_true, products_true, hazards_pred, products_pred):
        """
        Custom scoring function to compute the macro F1 score for hazards and products.

        Args:
            hazards_true: Ground truth labels for hazards.
            products_true: Ground truth labels for products.
            hazards_pred: Predicted labels for hazards.
            products_pred: Predicted labels for products.

        Returns:
            A float representing the combined macro F1 score.
        """
        f1_hazards = f1_score(hazards_true, hazards_pred, average='macro', zero_division=0)
        f1_products = f1_score(
            products_true[hazards_pred == hazards_true],
            products_pred[hazards_pred == hazards_true],
            average='macro',
            zero_division=0
        )
        return (f1_hazards + f1_products) / 2.

    # Example of calculating scores for Sub-Tasks (if needed):
    print(f"Score Sub-Task 1 - Logistic Regression Classifier: {compute_score(predictions_dict['hazard-category']['y_test'], predictions_dict['product-category']['y_test'], predictions_dict['hazard-category']['predictions'], predictions_dict['product-category']['predictions']):.3f}")
    print(f"Score Sub-Task 2 - Logistic Regression Classifier: {compute_score(predictions_dict['hazard']['y_test'], predictions_dict['product']['y_test'], predictions_dict['hazard']['predictions'], predictions_dict['product']['predictions']):.3f}")

    # save predictions to a new folder:
    os.makedirs('./submission/logreg/', exist_ok=True)
    testset_competition[['hazard-category', 'product-category', 'hazard', 'product']].to_csv('./submission/logreg.csv')
